Very nice final solution, less than 1s time in total!

In [1]:
with open("input22.txt", "r") as f:
    bricks = f.readlines()
bricks = [line.removesuffix("\n").replace("~",",").split(",") for line in bricks]
bricks = [tuple(map(int,(l[2],l[5],l[0],l[3],l[1],l[4]))) for l in bricks] #z first to sort by it

#order bricks by height
bricks = sorted(bricks)#, key=lambda b:b[2][0])

def xy_overlap(one_br, other_br):
    comp_v1, comp_v2 = one_br, other_br #(z,x,y) order
    if comp_v1[3]>=comp_v2[2] and comp_v1[2]<=comp_v2[3]: #x-component overlap
        if comp_v1[5]>=comp_v2[4] and comp_v1[4]<=comp_v2[5]: #y-component overlap
            return True
    return False

def fall(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks

def fall_singsupp(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    single_supporters = set() #set of bricks who are single supporters of some other brick
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
            supporters = {other_br for other_br in hovered_over if other_br[1]==hovered_ceil}
            if len(supporters)==1:
                single_supporters.add(supporters.pop())
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks, single_supporters

bricks, single_supporters = fall_singsupp(bricks)
len(bricks)- len(single_supporters) #,bricks, single_supporters

430

In [2]:
def xy_overlap(one_br, other_br):
    comp_v1, comp_v2 = one_br, other_br #(z,x,y) order
    if comp_v1[3]>=comp_v2[2] and comp_v1[2]<=comp_v2[3]: #x-component overlap
        if comp_v1[5]>=comp_v2[4] and comp_v1[4]<=comp_v2[5]: #y-component overlap
            return True
    return False
def fall(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks
def fall_supporters(bricks):#does the fall and makes dict of post-fall supports
    from copy import deepcopy
    bricks = deepcopy(bricks)
    supportdict = {}
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            supportdict.update({i: set()})
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
            supportdict.update({i: set()})
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
            supps = {other_br for other_br in hovered_over if other_br[1]==hovered_ceil}
            supportdict.update({i: {bricks.index(supp_br) for supp_br in supps}}) #brick floor height, then support
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks, supportdict

with open("input22.txt", "r") as f:
    bricks = f.readlines()
bricks = [line.removesuffix("\n").replace("~",",").split(",") for line in bricks]
bricks = [tuple(map(int,(l[2],l[5],l[0],l[3],l[1],l[4]))) for l in bricks] #z first to sort by it
bricks = sorted(bricks) #order bricks by pre-fall brick-foot height

bricks, support_dict = fall_supporters(bricks)
heights = [br[0] for br in bricks] #post-fall foot heights of every brick

fall_dict = {i:set() for i in range(len(bricks))} #who would fall if i disintegrated (all chain reaction)

for i in fall_dict:
    br_set = {i}
    while br_set!=set():
        max_height_brset = max({heights[br] for br in br_set})
        #set of parents of current set, unless someone in the current set is tall and its foot is below the others, in which case it remains
        br_set = {elem for br in br_set for elem in support_dict[br] if heights[br]==max_height_brset}.union({br for br in br_set if heights[br]< max_height_brset}) 
        if len(br_set)==1:
            fall_dict[min(br_set)].add(i) #min, but it's the only element
sum([len(s) for s in fall_dict.values()])

60558

In [33]:
#works, but very brute force, more than 3min30 to run
def xy_overlap(one_br, other_br):
    comp_v1, comp_v2 = one_br, other_br #(z,x,y) order
    if comp_v1[3]>=comp_v2[2] and comp_v1[2]<=comp_v2[3]: #x-component overlap
        if comp_v1[5]>=comp_v2[4] and comp_v1[4]<=comp_v2[5]: #y-component overlap
            return True
    return False
def fall(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks
def fall_singsupp(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    single_supporters = set() #set of bricks who are single supporters of some other brick
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
            supporters = [other_br for other_br in hovered_over if other_br[1]==hovered_ceil]
            if len(supporters)==1:
                single_supporters.add(supporters[0])
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks, single_supporters
def fall_and_count(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    count = 0
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
            count +=1
    return bricks, count

with open("input22.txt", "r") as f:
    bricks = f.readlines()
bricks = [line.removesuffix("\n").replace("~",",").split(",") for line in bricks]
bricks = [tuple(map(int,(l[2],l[5],l[0],l[3],l[1],l[4]))) for l in bricks] #z first to sort by it
bricks = sorted(bricks)#order bricks by height

bricks, single_supporters = fall_singsupp(bricks)
fallen_sum=0
for br in single_supporters:
    bricksminusone = bricks[:]
    bricksminusone.remove(br)
    bricksminusone, fall_count = fall_and_count(bricksminusone)
    fallen_sum += fall_count
fallen_sum

60558

In [87]:
#even slower than previous, canceled around the 4min mark
def xy_overlap(one_br, other_br):
    comp_v1, comp_v2 = one_br, other_br #(z,x,y) order
    if comp_v1[3]>=comp_v2[2] and comp_v1[2]<=comp_v2[3]: #x-component overlap
        if comp_v1[5]>=comp_v2[4] and comp_v1[4]<=comp_v2[5]: #y-component overlap
            return True
    return False
def fall(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks
def fall_singsupp(bricks):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    single_supporters = set() #set of bricks who are single supporters of some other brick
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
            supporters = [other_br for other_br in hovered_over if other_br[1]==hovered_ceil]
            if len(supporters)==1:
                single_supporters.add(supporters[0])
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks, single_supporters
def fall_supporters(bricks, supporters):
    from copy import deepcopy
    bricks = deepcopy(bricks)
    supporters = deepcopy(supporters)
    for i in range(len(bricks)):
        this_br = bricks[i]
        if this_br[0]==1:
            supporters.update({i:[]})
            continue
        hovered_over = [other_br for other_br in bricks[:i] if xy_overlap(this_br, other_br) and other_br[1]<this_br[0]]
        if hovered_over == []:
            hovered_ceil = 0
            supporters.update({i:[]})
        else:
            hovered_ceil = max({br[1] for br in hovered_over})
            supps = {other_br for other_br in hovered_over if other_br[1]==hovered_ceil}
            supporters.update({i: [bricks.index(supp_br) for supp_br in supps]})
        if hovered_ceil<this_br[0]-1:
            bricks[i] = (hovered_ceil+1, this_br[1]-this_br[0]+hovered_ceil+1, this_br[2], this_br[3], this_br[4], this_br[5])
    return bricks, supporters

#caching not useful: the same top brick might be called many times, but there might be a different dictionary each time, different falling patterns
def support_loss(disappeared, fallen, supporters):
    from copy import deepcopy
    supporters_dict = deepcopy(supporters)
    fallen = deepcopy(fallen)
    new_fallen = set()
    for br in supporters_dict:
        if disappeared in supporters_dict[br]:
            supporters_dict[br].remove(disappeared)
            if supporters_dict[br] == []:
                new_fallen.add(br)
                fallen.add(br)
    for br in new_fallen:
        extra_fallen, extra_supp_dict = support_loss(br, fallen, supporters_dict)
        fallen = fallen.union(extra_fallen)
        supporters_dict.update(extra_supp_dict)
    return fallen, supporters_dict

with open("input22.txt", "r") as f:
    bricks = f.readlines()
bricks = [line.removesuffix("\n").replace("~",",").split(",") for line in bricks]
bricks = [tuple(map(int,(l[2],l[5],l[0],l[3],l[1],l[4]))) for l in bricks] #z first to sort by it
bricks = sorted(bricks)#order bricks by height

supporters = {}
bricks, supporters = fall_supporters(bricks, supporters)
single_supporters = sorted(list({supporters[k][0] for k in supporters if len(supporters[k])==1})) #as a list of indices only

fallen_sum = 0
for i in range(len(bricks)):
    if i in single_supporters:
        fallen, _ = support_loss(i, set(), supporters)
        fallen_sum += len(fallen)
fallen_sum

KeyboardInterrupt: 